In [26]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [27]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/home/mikhail/Education/Semester_11/TransportNet/Stable Dynamic & Beckman/model.py'>

In [28]:
import numba
numba.__version__

'0.53.1'

In [29]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
from platform import python_version
print('python', python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
import graph_tool
print('graph_tool', graph_tool.__version__)

/usr/bin/python3
3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
python 3.6.9
numpy 1.19.5
pandas 1.1.4
graph_tool 2.43 (commit 9d41331e, Wed Jul 7 15:32:52 2021 +0200)


# Beckmann model paradox
$\rho = 1$ 
$\mu = 1$

In [30]:
beckmann_paradox_save = 'beckmann_paradox_results/'

In [31]:
net_name = 'BeckmanParadox_net.tntp'
trips_name = 'BeckmanParadox_trips.tntp'

MU = 1
RHO = 1

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

no_tax_model = md.Model(graph_data, graph_correspondences, 
                        total_od_flow, mu = MU, rho = RHO)

tax_model = md.ModelWithTaxes(graph_data, graph_correspondences, 
                              total_od_flow, mu = MU, rho = RHO)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,1.000000e-04,0.000001
1,1,True,3,True,1.000000e+20,45.000000
2,2,True,4,True,1.000000e+20,45.000000
3,3,True,4,True,1.000000e-04,0.000001
4,2,True,3,True,1.000000e+20,0.000001


In [32]:
max_iter = 100000

solver_kwargs = {'eps_abs': 0.3,
                 'max_iter': max_iter, 'stop_crit': 'dual_gap',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}

print('No tax calculation')
tic = time.time()
no_tax_result = no_tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

print('Tax calculation')
tic = time.time()
tax_result = tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

No tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 59 sec
Tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 57 sec


In [33]:
def form_list(arr):
     return ["{0:0.2f}".format(el) for el in arr]

def anarchy_cost(tax_result, no_tax_result):
    tax_vals = tax_result['taxes']
    tax_t = tax_result['times'] - tax_vals
    tax_f = tax_result['flows']
    no_tax_t = no_tax_result['times']
    no_tax_f = no_tax_result['flows']
    
    tax_total = np.dot(tax_t, tax_f)
    no_tax_total = np.dot(no_tax_t, no_tax_f)
    print("Tax flows:\t\t", *form_list(tax_f))
    print("Taxes costs:\t\t", *form_list(tax_vals))
    print("Tax total time:\t\t{:.2f}\n".format(tax_total))
    print("No tax flows:\t\t", *form_list(no_tax_f))
    print("No tax total time:\t{:.2f}".format(no_tax_total))
    print("\nAnarchy price:\t\t", no_tax_total / tax_total)

anarchy_cost(tax_result, no_tax_result)

Tax flows:		 2263.01 1736.99 1736.95 2263.05 526.07
Taxes costs:		 22.63 0.00 0.00 22.63 0.00
Tax total time:		257573.65

No tax flows:		 4000.00 -0.00 -0.00 4000.00 4000.00
No tax total time:	317840.27

Anarchy price:		 1.2339782250795903
